<a href="https://colab.research.google.com/github/Gail529/AES/blob/main/automated_essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
 
import re
import string
import warnings
warnings.filterwarnings("ignore")  
from gensim.models import Word2Vec
 
 
#essay preprocessing 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#training the neural network 
from tensorflow.keras.layers import Embedding
from keras.models import Sequential, load_model, model_from_config
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Flatten
import keras.backend as K

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
essay_data= pd.read_excel('/content/training_set_rel3.xls',usecols=['essay_id','essay','domain1_score'])
essay_data.head(2)

,essay_id,essay,domain1_score
0,1,"Dear local newspaper, I think effects computer...",8.0
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0


In [4]:
x=essay_data['essay']
y=essay_data['domain1_score']
len(x) 

12978

In [5]:
def sent_tokenize(text):
    stripped_essay = text.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
        words = clean_sentence.lower().split()
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        tokenized_sentences.append(words)

    return tokenized_sentences


In [6]:
cleaned_essays=[]
for i in range(0,len(x)):
    essay=x[i]
    cleaned_essays.append(sent_tokenize(essay))
    

In [7]:
sentences=[]
for essay in x:
    sentences += sent_tokenize(essay)

In [ ]:
sentences[100]

['result', 'computers', 'would', 'improve', 'way', 'learn']

In [8]:
text_dim = 300
print("Training Word2Vec model...")
wordvec_model = Word2Vec(sentences, size=text_dim, window=5, min_count=3, workers=4, sg=1)
print("Word2Vec model created.")
print("%d unique words represented by %d dimensional vectors" % (len(wordvec_model.wv.vocab), text_dim))
wordvec_model.save('wordvec_model')
print("Word2Vec model saved.")

Training Word2Vec model...
Word2Vec model created.
13134 unique words represented by 300 dimensional vectors
Word2Vec model saved.


In [9]:
def clean_essays(essay):
    clean_essay= re.sub("[^a-zA-Z]", " ",essay)
    words = essay.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

In [10]:
cleaned_essays = []
for essay in x:
        cleaned_essays.append(clean_essays(essay))

In [11]:
def create_average_vec(essay):
    average = np.zeros((text_dim,), dtype='float32')
    num_words = 0.
    for word in essay:
        if word in wordvec_model.wv.vocab:
            average = np.add(average, wordvec_model.wv[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

In [12]:
cleaned_vec = np.zeros((len(x),text_dim), dtype="float32")  
for i in range(len(cleaned_essays)):
    cleaned_vec[i] = create_average_vec(cleaned_essays[i])

print("Word vectors for all essays in the training data set are of shape:", cleaned_vec.shape)


Word vectors for all essays in the training data set are of shape: (12978, 300)


In [13]:
data_split=0.8
train_samples=int(len(cleaned_vec)*data_split)

x_train = np.array(cleaned_vec[:train_samples])
x_test = np.array(cleaned_vec[train_samples:])
y_train = np.array(y[:train_samples])
y_test = np.array(y[train_samples:])

print("x_train" ,len(x_train))
print("x_test" ,len(x_test))
print("y_train" ,len(y_train))
print("y_test" ,len(y_test))

x_train 10382
x_test 2596
y_train 10382
y_test 2596


In [ ]:
cleaned_essays[89]

In [14]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [15]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    

lstm_model = get_model()
lstm_model.fit(x_train, y_train, batch_size=64, epochs=50)
y_pred = lstm_model.predict(x_test)
lstm_model.save('final_lstm.h5')

# Round y_pred to the nearest integer.
y_pred = np.around(y_pred)

# Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
result = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result))



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 300)            721200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
163/163 [==============================] - 9s 26ms/step - loss: nan - mae: nan
Epoch 2/50
163/163 [==============================] - 4s 27ms/step - loss: nan - mae: nan
Epoch 3/50
163/163 [==============================] - 4s 26ms/step -

KeyboardInterrupt: ignored